In [4]:
import numpy as np
import pandas as pd
df = pd.DataFrame({'a': np.random.randn(1000),
                   'b': np.random.randn(1000),
                   'N': np.random.randint(100, 1000, (1000)),
                   'x': 'x'})

In [5]:
def f(x):
    return x * (x - 1)

In [6]:
def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [7]:
%timeit df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

10 loops, best of 3: 144 ms per loop


In [8]:
%prun -l 4 df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

In [9]:
%load_ext Cython

In [11]:
%%cython
def f_plain(x):
    return x * (x - 1)

def integrate_f_plain(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_plain(a + i * dx)
    return s * dx

/home/cma/.envs/venv-py/local/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/cma/.cache/ipython/cython/_cython_magic_ff51103303268416f35fb21bb0537e28.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [12]:
timeit df.apply(lambda x: integrate_f_plain(x['a'], x['b'], x['N']), axis=1)

10 loops, best of 3: 92 ms per loop


In [13]:
%%cython
cdef double f_typed(double x) except? -2:
    return x * (x - 1)

cpdef double integrate_f_typed(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_typed(a + i * dx)
    return s * dx

/home/cma/.envs/venv-py/local/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/cma/.cache/ipython/cython/_cython_magic_12b52322e008b01f9bcd80ccdccde6f8.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [14]:
timeit df.apply(lambda x: integrate_f_typed(x['a'], x['b'], x['N']), axis=1)

10 loops, best of 3: 24.2 ms per loop


In [15]:
%%cython
cimport numpy as np
import numpy as np
cdef double f_typed(double x) except? -2:
    return x * (x - 1)

cpdef double integrate_f_typed(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_typed(a + i * dx)
    return s * dx

cpdef np.ndarray[double] apply_integrate_f(np.ndarray col_a, np.ndarray col_b,
                                           np.ndarray col_N):
    assert (col_a.dtype == np.float 
            and col_b.dtype == np.float and col_N.dtype == np.int)
    cdef Py_ssize_t i, n = len(col_N)
    assert (len(col_a) == len(col_b) == n)
    cdef np.ndarray[double] res = np.empty(n)
    for i in range(len(col_a)):
        res[i] = integrate_f_typed(col_a[i], col_b[i], col_N[i])
    return res

/home/cma/.envs/venv-py/local/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/cma/.cache/ipython/cython/_cython_magic_b02dc5643200f38c2398cba9df606504.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [16]:
%timeit apply_integrate_f(df['a'].values, df['b'].values, df['N'].values)

1000 loops, best of 3: 899 µs per loop


In [21]:
%%cython
cimport cython 
cimport numpy as np
import numpy as np
cdef double f_typed(double x) except? -2:
    return x * (x - 1)

cpdef double integrate_f_typed(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_typed(a + i * dx)
    return s * dx
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef np.ndarray[double] apply_integrate_f_wrap(np.ndarray col_a, np.ndarray col_b,
                                           np.ndarray col_N):
    assert (col_a.dtype == np.float 
            and col_b.dtype == np.float and col_N.dtype == np.int)
    cdef Py_ssize_t i, n = len(col_N)
    assert (len(col_a) == len(col_b) == n)
    cdef np.ndarray[double] res = np.empty(n)
    for i in range(len(col_a)):
        res[i] = integrate_f_typed(col_a[i], col_b[i], col_N[i])
    return res

/home/cma/.envs/venv-py/local/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/cma/.cache/ipython/cython/_cython_magic_3233d8010b3d579163fde2fea8cbb72b.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [22]:
%timeit apply_integrate_f_wrap(df['a'].values, df['b'].values, df['N'].values)

1000 loops, best of 3: 892 µs per loop
